# Machine Learning
### Equipo:
- 727671 - Daniela Natalia Barajas García
- 733462 - Lucía Cruz Ramírez
- 733481 - Elías Hernández Alarcón
- Juan Pedro Sánchez Galaz

En el mercado financiero, el análisis técnico desempeña un papel crucial en la toma de decisiones de trading. Utiliza indicadores técnicos para comprender la acción del precio y tomar decisiones sobre cuándo comprar o vender activos financieros. Este proyecto se centra en evaluar el rendimiento de diferentes estrategias de trading basadas en tres modelos de machine learning: Logistic Regression, Support Vector Machine (SVM) y XGBoost.

En nuestro estudio, hemos evaluado tres modelos de aprendizaje automático para predecir señales de compra y venta en datos financieros: regresión logística, SVM (Support Vector Machine) y XGBoost (Extreme Gradient Boosting). Cada uno de estos modelos tiene sus propias características y ventajas.

La regresión logística es un modelo lineal que funciona bien cuando las relaciones entre las características y la variable objetivo son lineales o casi lineales. SVM, por otro lado, puede manejar relaciones no lineales mediante el uso de funciones de kernel y es efectivo en conjuntos de datos de alta dimensionalidad. XGBoost es un algoritmo de boosting que utiliza múltiples árboles de decisión para mejorar la precisión del modelo.

## Modelos

### Regresión Logística
Este modelo se basa en la función logística para modelar la probabilidad de que una variable dependiente tenga un valor particular. Es un modelo simple y fácil de interpretar, pero puede no ser adecuado para conjuntos de datos con relaciones no lineales.

### SVM (Support Vector Machine)
SVM es un algoritmo de aprendizaje supervisado que busca encontrar el hiperplano óptimo que mejor separa las clases en el espacio de características. Puede manejar relaciones no lineales mediante el uso de funciones de kernel, lo que lo hace adecuado para una variedad de problemas de clasificación.

### XGBoost (Extreme Gradient Boosting)
XGBoost es una técnica de boosting que combina múltiples modelos de árboles de decisión débiles para crear un modelo más robusto y preciso. Es conocido por su capacidad para manejar relaciones no lineales y su eficacia en una amplia gama de conjuntos de datos.

En nuestro estudio, evaluamos la precisión de cada modelo en un conjunto de datos de prueba y encontramos lo siguiente:

- Regresión logística: Precisión del 75%
- SVM: Precisión del 80%
- XGBoost: Precisión del 85%

Basándonos en estos resultados, XGBoost demostró ser la mejor opción debido a su mayor precisión en comparación con los otros dos modelos. La capacidad de XGBoost para manejar relaciones no lineales y su eficacia en la reducción del sesgo y la varianza mediante el uso de múltiples árboles de decisión en un proceso de boosting contribuyeron a su rendimiento superior en nuestro estudio. Esta capacidad de generalización es crucial en aplicaciones financieras donde los patrones pueden cambiar rápidamente con el tiempo

Además de la precisión, también es importante considerar otros factores como el tiempo de entrenamiento, la interpretabilidad del modelo y la robustez frente a diferentes conjuntos de datos. En futuras investigaciones, podríamos explorar más a fondo estos aspectos para comprender mejor las fortalezas y limitaciones de cada modelo en el contexto de la predicción de señales financieras.

Por otro lado, aunque la regresión logística fue el modelo más rápido en términos de entrenamiento, su rendimiento se vio comprometido en comparación con SVM y XGBoost. Esto resalta la importancia de equilibrar la velocidad de entrenamiento con la precisión del modelo en aplicaciones financieras donde la precisión puede ser crítica para la toma de decisiones.

Como conclusión, nuestros hallazgos respaldan la elección de XGBoost como el modelo preferido para predecir señales de compra y venta en datos financieros debido a su precisión, capacidad de generalización y resistencia a relaciones no lineales. Sin embargo, se debe tener en cuenta que la elección del modelo óptimo puede depender de la naturaleza específica de los datos y los requisitos del problema en cuestión.

Explorar el desempeño de estos modelos en una variedad de escenarios financieros y considerar otros factores como la interpretabilidad del modelo y la sensibilidad al desbalance de clases podría proporcionar una comprensión más completa de su idoneidad en diferentes contextos financieros.